In [ ]:
# Put colored Images in colored folder
# Put grayscale Images in images folder
# JPEG_graphs contains the graphs
# JPEG_analysis contains the before and after images
# JPEG contains the textfiles with the encoded data

In [ ]:
# importing modules
import numpy as np
import matplotlib.pyplot as plt
import cv2 as cv
import math
import os
import json

Analysis Functions

In [ ]:
def getPSNR(original, modified):
    if(len(original.shape)==3):
        # Convert the image to RGB for saving
        original = cv.cvtColor(original, cv.COLOR_YCR_CB2BGR)
    
    if(len(modified.shape)==3):
        # Convert the image to RGB for saving
        modified = modified.astype('uint8')
        modified = cv.cvtColor(modified, cv.COLOR_YCR_CB2BGR)
        
    mse = np.mean((original - modified) ** 2)
    if(mse == 0):  # MSE is zero means no noise is present in the signal .
                 
        return -1
    
    max_pixel = 255.0

    psnr = 20 * math.log10(max_pixel / math.sqrt(mse))
    return psnr

In [ ]:
# calculating RMSE
def CalculateRMSE(originalImage, reconstructedImage):
    if(len(originalImage.shape)==3):
        # Convert the image to RGB for saving
        originalImage = cv.cvtColor(originalImage, cv.COLOR_YCR_CB2BGR)
    
    if(len(reconstructedImage.shape)==3):
        # Convert the image to RGB for saving
        reconstructedImage = reconstructedImage.astype('uint8')
        reconstructedImage = cv.cvtColor(reconstructedImage, cv.COLOR_YCR_CB2BGR)
        
    rmse = math.sqrt(np.mean((originalImage - reconstructedImage) ** 2))
    return rmse

In [ ]:
def CalculateCompressionRatio(img, encodedSequence, blockSize):
    # bits in originalImage/bits in encodedSequence
    rows, cols = img.shape[0], img.shape[1]    
    # calculating bits in original image
    if(len(img.shape)==3):
        originalBits = rows*cols*3
    else:
        originalBits = rows*cols
    compressedBits = 0
    
    if(blockSize==-1):
        compressedBits = len(encodedSequence)
        return originalBits/compressedBits
    
    for i in encodedSequence:
        compressedBits += len(i)
    
    return originalBits/compressedBits

In [ ]:
def ResizeQMatrix(QMatrix,blockSize):
    QMatrix = np.float32(QMatrix)
    QMatrix = cv.resize(QMatrix, (blockSize, blockSize), interpolation = cv.INTER_AREA)
    QMatrix = np.int32(QMatrix)
    return QMatrix

In [ ]:
def fixImageDimensions(img, blockSize):
    rows, cols = img.shape[0], img.shape[1]

    if(len(img.shape)==3):
        b,g,r = cv.split(img)
        if(rows%blockSize != 0):
            newRows = ((rows//blockSize)+1)*blockSize
            r = np.pad(img, ((0, newRows-rows), (0, 0)), mode='constant')
            g = np.pad(img, ((0, newRows-rows), (0, 0)), mode='constant')
            b = np.pad(img, ((0, newRows-rows), (0, 0)), mode='constant')
   
        if(cols%blockSize != 0):
            newCols = ((cols//blockSize)+1)*blockSize
            r = np.pad(img, ((0,0), (0, newCols-cols)), mode='constant')
            g = np.pad(img, ((0,0), (0, newCols-cols)), mode='constant')
            b = np.pad(img, ((0,0), (0, newCols-cols)), mode='constant')

        img = cv.merge((b,g,r))

    else:
        if(rows%blockSize != 0):
            newRows = ((rows//blockSize)+1)*blockSize
            
            img = np.pad(img, ((0, newRows-rows), (0, 0)), mode='constant')
            

        if(cols%blockSize != 0):
            newCols = ((cols//blockSize)+1)*blockSize
            img = np.pad(img, ((0,0), (0, newCols-cols)), mode='constant')
    
    return img

In [ ]:
# steps for jpeg compression
# 1. convert image to YUV
# 2. downsample Cb and Cr
# 3. do img = img-128, to each channel
# 4. apply DCT to Y
# 5. quantize Y
# 6. do encoding on the values obtained


# Getting back the compressed image from encoded values
# 0. do decoding on the encoded values
# 1. dequantize Y
# 2. apply inverse DCT to Y
# 3. upsample Cb and Cr
# 4. convert image back to RGB


Compress

Helper Functions


In [ ]:
# function that performs dct on an image
def dct(img):
    
    # performing dct on image
    if(len(img.shape)==3):

        r,g,b=cv.split(img)
        r = r.astype('float32')
        g = g.astype('float32')
        b = b.astype('float32')
        r = cv.dct(r)
        g = cv.dct(g)
        b = cv.dct(b)
        r = r.astype('int32')
        g = g.astype('int32')
        b = b.astype('int32')
        newImg = cv.merge((r,g,b))
        newImg = newImg.astype('int32')

    else:
        imf = img.astype('float32')
        
        newImg =  cv.dct(imf)
        newImg = newImg.astype('int32')
    return newImg



In [ ]:
# img = img - 128 function
def ShiftImage(img):
    img = img.astype('int32')
    img = img - 128
    return img


In [ ]:
def quantizeImg(img, QMatrix):
    # quantizing image
    if(len(img.shape)==3):
        r,g,b = cv.split(img)
        r = np.divide(r,QMatrix)
        g = np.divide(g,QMatrix)
        b = np.divide(b,QMatrix)
        img = cv.merge((r,g,b))
        return img
    else:
        img  = np.divide(img, QMatrix)
        img = img.astype('int32')
        return img
    

In [ ]:
def fillDecodeArray(rows, cols, blockSize, img, encoded, coefficientNum):

    # encoded is a list of encoded blocks -> list of lists
    for i in range(0, rows, blockSize):
        for j in range(0, cols, blockSize):
            # applying zigzag scan to each block
            encoded.append(zigzagScan(img[i:i+blockSize, j:j+blockSize], blockSize, coefficientNum))
    
    
    return encoded

Compression Functions

In [ ]:

# main function 
def Compress(n, img):
    rows, cols = img.shape[0], img.shape[1]

    # shift image by 128
    img = ShiftImage(img)

    # now we have the shifted image, so now we can break the image into nxn and apply DCT to it
    for i in range(0, rows, n): # start from 0 to rows and take the step size of n
        for j in range(0, cols, n): # start from 0 to cols and take the step size of n
            # applying dct to each block
            img[i:i+n, j:j+n] = dct(img[i:i+n, j:j+n])
    
    # now we have the image with DCT applied to it, so now we can quantize it
    # again break the image into 8x8 blocks and apply quantization to it
    for i in range(0,rows, n):
        for j in range(0,cols, n):
            # applying quantization to each block
            img[i:i+n, j:j+n] = quantizeImg(img[i:i+n, j:j+n], QMatrix)
    
    return img

Encoding Functions


In [ ]:
def zigzagScan(img, n, coeffNum):

    # encoded = []
    encoded = np.array([])
    for i in range(n):
        if i % 2 == 0:
            for j in range(n):
                encoded = np.append(encoded, img[i][j])
        else:
            for j in range(n - 1, -1, -1):
                encoded = np.append(encoded, img[i][j])
                # encoded.append(img[i][j])
    
    numberOfPixels = n**2
    if(coeffNum==-1):
        # now we remove the redundant coefficients/zeros at the end
        while(len(encoded)>0):
            if(encoded[-1]==0):
                encoded = encoded[:-1]
            else:
                break
    elif(coeffNum<=n*n):
        encoded[coeffNum:numberOfPixels-1] =  0
        # now we remove the redundant coefficients/zeros at the end
        while(len(encoded)>0):
            if(encoded[-1]==0):
                encoded = encoded[:-1]
            else:
                break
    else:
        print("Wrong Number of Coefficients")
        pass

    return encoded


In [ ]:
def JPEG_encoder(img, blockSize, coefficientNum, imagefileName):
    img = Compress(blockSize, img)
    

    rows, cols = img.shape[0], img.shape[1]
    # now we have the quantized image, so now we can do encoding on it
    # we can use zigzag scan to do encoding
    #for every nxn block, we will do zigzag scan
    # encoded = np.array([])
    encoded = [] # list of encoded blocks -> list of lists
    
    if(len(img.shape)==3): # color image
        r,g,b = cv.split(img)

        # fill the encoded array with r encoded values, g, and then b encoded values
        encoded = fillDecodeArray(rows, cols, blockSize, r, encoded, coefficientNum)
        encoded = fillDecodeArray(rows, cols, blockSize, g, encoded, coefficientNum)
        encoded = fillDecodeArray(rows, cols, blockSize, b, encoded, coefficientNum)

    else: # grayscale image
        
        for i in range(0, rows, blockSize):
            for j in range(0, cols, blockSize):
                # applying zigzag scan to each block
                encoded.append(zigzagScan(img[i:i+blockSize, j:j+blockSize], blockSize, coefficientNum))


    # convert the encoded list to numpy array    
    encoded = np.array(encoded, dtype=object)

    # save the encoded values to a csv file
    saveDestination = 'JPEG/encoded_'+imagefileName+'.csv'
    np.savetxt(saveDestination,encoded, delimiter=',', fmt='%s')

    return saveDestination, encoded


DeCompressing

Helper Functions

In [ ]:
def idct(img):

    if(len(img.shape)==3):

        r,g,b=cv.split(img)
        r = r.astype('float32')
        g = g.astype('float32')
        b = b.astype('float32')
        r = cv.idct(r)
        g = cv.idct(g)
        b = cv.idct(b)
        r = r.astype('int32')
        g = g.astype('int32')
        b = b.astype('int32')

        newImg = cv.merge((r,g,b))
        return newImg
    else:
        imf = img.astype('float32')
        imf =  cv.idct(imf)
        return imf.astype('int32')
    

In [ ]:
def dequantizeImg(img, QMatrix):
    # dequantizing image

    if(len(img.shape)==3):
        r,g,b = cv.split(img)
        r = np.multiply(r,QMatrix)
        g = np.multiply(g,QMatrix)
        b = np.multiply(b,QMatrix)
        img = cv.merge((r,g,b))
    else:
        img = np.multiply(img, QMatrix)
    
    return img

In [ ]:
def ShiftPositive(img):
    img = img.astype('int32')
    img = img + 128
    return img

In [ ]:
def reconstructBlockFromEncoded(rows, cols, blockSize, encodedArray, offset):
    reconstructedImageFromEncoded = np.zeros((rows, cols), dtype='int32')
    index = 0
    for i in range(0, rows, blockSize):
        for j in range(0, cols, blockSize):
            # applying zigzag scan to each block
            # we find the start index for from where we should start reading the values in the encodedArray, because the encodedArray contains all the values of all the blocks concateanated
            reconstructedImageFromEncoded[i:i+blockSize, j:j+blockSize] = ZigZacPutBack(encodedArray[offset+index], blockSize)
            index+=1
            
    return reconstructedImageFromEncoded

Decompression Functions


In [ ]:
def Decompress(n, img):
    rows, cols = img.shape[0], img.shape[1]

    # first we dequantize the image by breaking the image into several nxn blocks
    for i in range(0,rows, n):
        for j in range(0,cols, n):
            # applying dequantization to each block
            img[i:i+n, j:j+n] = dequantizeImg(img[i:i+n, j:j+n], QMatrix)
    
    # now we apply idct on each block
    for i in range(0, rows, n):
        for j in range(0, cols, n):
            # applying idct to each block
            img[i:i+n, j:j+n] = idct(img[i:i+n, j:j+n])
    
    # now we shift the image back to positive values
    img = ShiftPositive(img)

    return img

Decoding Functions


In [ ]:
def ZigZacPutBack(encodedArray, blockSize):
    imgSegment = np.zeros((blockSize, blockSize), dtype='int32')
    # print(encodedArray)
    size = len(encodedArray)
    correctEncoded = np.zeros((blockSize**2), dtype='int32')
    correctEncoded[:size] = encodedArray[:]
    encodedArray = correctEncoded
    
    index = 0
    for i in range(blockSize):
        
        if(i%2==0):
            for j in range(blockSize):
                imgSegment[i][j] = encodedArray[index]
                index+=1
        else:
            for j in range(blockSize-1, -1, -1):
                imgSegment[i][j] = encodedArray[index]
                index+=1
                
    return imgSegment


In [ ]:
def JPEG_decoder(encodedArray,img, blockSize):
    
    # reconstructing the image
    rows, cols = img.shape[0], img.shape[1]
    # this is to get the total number of blocks in 1 layer so that we can encode the array into these many lists
    blocksIn1Layer = rows*cols//(blockSize**2)
    
    # we have the encodedArray and block size, so we can easily reconstruct the image matrix
    if(len(img.shape)==3): # color image
        
        r,g,b = cv.split(img)
        
        offset = 0
        r = reconstructBlockFromEncoded(rows, cols, blockSize, encodedArray, offset)
        offset += blocksIn1Layer
        g = reconstructBlockFromEncoded(rows, cols, blockSize, encodedArray, offset)
        offset += blocksIn1Layer
        b = reconstructBlockFromEncoded(rows, cols, blockSize, encodedArray, offset)

        reconstructedImage = cv.merge((r,g,b))

    else: # grayscale image
        reconstructedImage = np.zeros((rows, cols), dtype='int32')
        index = 0
        for i in range(0, rows, blockSize):
            for j in range(0, cols, blockSize):
                # applying zigzag scan to each block
                reconstructedImage[i:i+blockSize, j:j+blockSize] = ZigZacPutBack(encodedArray[index], blockSize)
    
    RMSE = CalculateRMSE(img, reconstructedImage)
    PSNR = getPSNR(img, reconstructedImage)
    CompressionRatio = CalculateCompressionRatio(img, encodedArray, blockSize)
    reconstructedImage = Decompress(blockSize, reconstructedImage)
    
    return reconstructedImage, RMSE, PSNR, CompressionRatio



CODE TO RECONSTRUCT THE IMAGE


Quantization Matrices

In [ ]:
# QMatrix for 8x8 blocks
QMatrix = np.array([[16, 11, 10, 16, 24, 40, 51, 61], 
                    [12, 12, 14, 19, 26, 58, 60, 55],
                    [14, 13, 16, 24, 40, 57, 69, 56],
                    [14, 17, 22, 29, 51, 87, 80, 62],
                    [18, 22, 37, 56, 68, 109, 103, 77],
                    [24, 35, 55, 64, 81, 104, 113, 92],
                    [49, 64, 78, 87, 103, 121, 120, 101],
                    [72, 92, 95, 98, 112, 100, 103, 99
                     ]])


In [ ]:
image_folderName = 'images/'
image_filename = 'lena.tif' # example, change the image you wish to compress and decompress here
img = cv.imread(image_folderName+image_filename, cv.IMREAD_GRAYSCALE) #reading grayscale images currently

if(len(img.shape)==3):
    img = cv.cvtColor(img, cv.COLOR_BGR2YCR_CB)

blockSize = 8
QMatrix = ResizeQMatrix(QMatrix,blockSize)

NumberOfCoefficientsToSend = -1 # -1 means send all the coefficients

img = fixImageDimensions(img, blockSize)
saveDestination, encoded = JPEG_encoder(img, blockSize, NumberOfCoefficientsToSend, image_filename)
reconstructedImage,RMSE ,PSNR,CompressionRatio = JPEG_decoder(encoded, img, blockSize)
plt.imshow(img,cmap='gray')

In [ ]:
image_folderName = 'colored/'
image_filename = 'kodim01.png' # example, change the image you wish to compress and decompress here
img = cv.imread(image_folderName+image_filename, cv.IMREAD_COLOR) #reading grayscale images currently

if(len(img.shape)==3):
    img = cv.cvtColor(img, cv.COLOR_BGR2YCR_CB)

blockSize = 8
QMatrix = ResizeQMatrix(QMatrix,blockSize)

NumberOfCoefficientsToSend = -1 # -1 means send all the coefficients

img = fixImageDimensions(img, blockSize)
saveDestination, encoded = JPEG_encoder(img, blockSize, NumberOfCoefficientsToSend, image_filename)
reconstructedImage,RMSE ,PSNR,CompressionRatio = JPEG_decoder(encoded, img, blockSize)
if(len(img.shape)==3):
        # Convert the image to RGB for saving
        img = cv.cvtColor(img, cv.COLOR_YCR_CB2RGB)
        # convert the decompressed image to RGB for showing
        reconstructedImage = reconstructedImage.astype('uint8')
        reconstructedImage = cv.cvtColor(reconstructedImage, cv.COLOR_YCR_CB2RGB)

plt.figure(figsize=[16,5])
plt.suptitle('JPEG Compression')
plt.subplot(1,2,1);plt.imshow(img);plt.title('Original Image')
plt.subplot(1,2,2);plt.imshow(reconstructedImage);plt.title(f'Reconstructed Image (RMSE={RMSE:.2f}, PSNR={PSNR:.2f}, Compression Ratio={CompressionRatio:.2f})')